In [2]:
# Import necessary packages
# Ensure installation of nltk package in conda environment
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Add directory above current directory to path so we can import our pre-built package
import sys; sys.path.insert(0, '../..')
from entity_disambiguation.preprocessing import process_input, normalize_text

# Import NLP stopwords
from nltk.corpus import stopwords

# Import progress bar package
from tqdm import tqdm

In [3]:
try: 
    stop = stopwords.words('english')
except LookupError:
    # Download stopwords if it's your first time
    import nltk
    nltk.download('stopwords')

## 1. Process Aida-Conll-Yago (ACY) dataset into train/test split

In [4]:
# relative path to ACY dataset, should be the same for all users within `entity-disambiguation` repository
acy_path = '../../data/aida-conll-yago-dataset/'

# process_input() takes path location where you've stored AIDA-YAGO2-DATASET.tsv file
# Transforms tsv file into train, test split
# x are words, y are indices
train_x, train_y, test_x, test_y = process_input(acy_path, train = .8)

In [5]:
# Display shape of output train data
display((train_x.shape, train_y.shape))
display((test_x.shape, test_y.shape))

((17805, 2), (17805,))

((4452, 2), (4452,))

In [6]:
# Display x preview
train_x.head(10)

,token,in_between_word_count
0,german,2
1,british,3
2,brussels,6
3,european,3
4,commission,0
5,german,6
6,british,5
7,germany,15
8,european,4
9,union,0


In [7]:
# Display y preview
train_y[0:3]

array(['11867', '31717', '3708'], dtype='<U8')

In [8]:
# Save train x/y into numpy arrays
x_np = train_x.token.values
y_np = np.array(train_y, dtype = 'int64')

### Prepare normalized KWNLP

In [10]:
# Provide directory path for KWNLP data
# Should be same for all users of `entity-disambiguation` repository
kwnlp_path = '../../data/kwnlp'

# Load article data
article_df = pd.read_csv(os.path.join(kwnlp_path, 'kwnlp-enwiki-20200920-article.csv'))

# Load anchor target counts data
anchor_df = pd.read_csv(os.path.join(kwnlp_path, 'kwnlp-enwiki-20200920-anchor-target-counts.csv'))

In [11]:
# Display article preview
article_df.head()

,page_id,item_id,page_title,views,len_article_chars,len_intro_chars,in_link_count,out_link_count,tmpl_good_article,tmpl_featured_article,tmpl_pseudoscience,tmpl_conspiracy_theories,isa_Q17442446,isa_Q14795564,isa_Q18340514
0,12,6199,Anarchism,35558,40449,409,3826,371,1,0,0,0,0,0,0
1,25,38404,Autism,40081,47659,419,2313,309,0,1,0,0,0,0,0
2,39,101038,Albedo,10770,18766,293,3090,115,0,0,0,0,0,0,0
3,290,9659,A,29398,9538,609,173,149,0,0,0,0,0,0,0
4,303,173,Alabama,46680,74276,369,11864,744,0,0,0,0,0,0,0


In [12]:
# Display anchor preview
anchor_df.head()

,anchor_text,target_page_id,count
0,United States,3434750,152451
1,World War II,32927,133668
2,India,14533,112069
3,France,5843419,109669
4,footballer,10568,101027


In [13]:
# Copy anchor_df to new dataframe
at_count_df = anchor_df.copy()

# Normalize anchor_text (lower-case, strip whitespace)
at_count_df["normalized_anchor_text"] = at_count_df["anchor_text"].apply(normalize_text)

# Return all anchor_texts that are non-zero, non-null
at_count_df = at_count_df.loc[at_count_df['normalized_anchor_text'].str.len() > 0, :]

print(len(at_count_df))
at_count_df.head(3)

15269229


,anchor_text,target_page_id,count,normalized_anchor_text
0,United States,3434750,152451,united states
1,World War II,32927,133668,world war ii
2,India,14533,112069,india


Inner join anchor-target data (mention to linked entity) with Wikipedia page article data. This lets us collate stats like page views with the target (entity) of mentions. Page views serves as another baseline model by selecting the page/entity that is most viewed for that anchor text.

In [14]:
# Merge at_count and article stats dataframes
at_count_df = pd.merge(
    at_count_df,
    article_df,
    how="inner",
    left_on="target_page_id",
    right_on="page_id")

# Rename columns for clarity
at_count_df = at_count_df.rename(columns={
    'title': 'target_page_title',
    'item_id': 'target_item_id',
    'views': 'target_page_views',
    'count': 'anchor_target_count',
    'page_title': 'target_page_title'})

# Specify column ordering
at_count_df = at_count_df[[
    "normalized_anchor_text",
    "target_page_id",
    "target_item_id",
    "target_page_title",
    "target_page_views",
    "anchor_target_count"]]

# Display preview
at_count_df.head(3)

,normalized_anchor_text,target_page_id,target_item_id,target_page_title,target_page_views,anchor_target_count
0,united states,3434750,30,United_States,460156,152451
1,american,3434750,30,United_States,460156,65722
2,usa,3434750,30,United_States,460156,8559


Drop NaNs. These have been encoded as `string` through the text normalisation previously.

In [15]:
# Drop NaNs
len_orig = len(at_count_df)
at_count_df = at_count_df.loc[at_count_df['normalized_anchor_text'] != 'nan']
print('Dropped rows:', len_orig - len(at_count_df))

Dropped rows: 3596


## Develop Wikipedia2Vec Model

We will create a normalized anchor text dataset from KWNLP. We will then use the pre-trained word embeddings provided via [Wikipedia2Vec](https://wikipedia2vec.github.io/wikipedia2vec/) to calculate a similarity score between a mention in ACY and a normalized anchor text in KWNLP.

In [16]:
# Import package
from wikipedia2vec import Wikipedia2Vec

In [17]:
# Load unzipped pkl file
wiki2vec = Wikipedia2Vec.load("../../embeddings/enwiki_20180420_100d.pkl")

In [29]:
# Experiment with returning words from anchor texts
for word in at_count_df['normalized_anchor_text'][:5]:
    print("Anchor Text: ", word)
    found_word = wiki2vec.get_word(word)
    print("Found Word: ", found_word)
    if found_word is None:
        continue
    else:
        similar_word = wiki2vec.most_similar(wiki2vec.get_word(word), 5)
        print("Similar Word: ", similar_word)
    print("**************************")
    

Anchor Text:  united states
Found Word:  None
Anchor Text:  american
Found Word:  <Word american>
Similar Word:  [(<Word american>, 1.0), (<Word african>, 0.6946235), (<Word canadian>, 0.6769873), (<Word africanamerican>, 0.67509526), (<Word caribbeanist>, 0.6693679)]
**************************
Anchor Text:  usa
Found Word:  <Word usa>
Similar Word:  [(<Word usa>, 1.0000001), (<Word finnfest>, 0.7673373), (<Word priceton>, 0.74322855), (<Word mddtusa>, 0.72683185), (<Word hanahou>, 0.7265224)]
**************************
Anchor Text:  u.s.
Found Word:  None
Anchor Text:  us
Found Word:  <Word us>
Similar Word:  [(<Word us>, 1.0), (<Word 20px25px>, 0.72674584), (<Word truckstops>, 0.6859466), (<Word 39billion>, 0.68373144), (<Word 2004dollars>, 0.67024976)]
**************************


In [36]:
# Experiment with returning entities from anchor texts
for word in at_count_df['normalized_anchor_text'][:5]:
    print("Anchor Text: ", word)
    found_entity = wiki2vec.get_entity(word)
    print("Found Entity: ", found_entity)
#     if found_entity is None:
#         continue
#     else:
#         similar_word = wiki2vec.most_similar(wiki2vec.get_word(word), 5)
#         print("Similar Word: ", similar_word)
    print("**************************")
    

Anchor Text:  united states
Found Entity:  None
**************************
Anchor Text:  american
Found Entity:  None
**************************
Anchor Text:  usa
Found Entity:  None
**************************
Anchor Text:  u.s.
Found Entity:  None
**************************
Anchor Text:  us
Found Entity:  None
**************************


In [37]:
# Our normalizing function has actually prevented wiki2vec from correctly identifying entities
# Below, capitalized 'United States' returns correct entity whereas lowercased 'united states' does not
print(wiki2vec.get_entity('United States'))
print(wiki2vec.get_entity('united states'))

<Entity United States>
None


#### Challenges & Pivot

It appears unlikely to be able to use Wikipedia2Vec to automatically link anchor texts to Entities. Instead, let's try to use Gensim and the word similarity score or distance metric functionality.

In [39]:
# Import gensim
from gensim.models import KeyedVectors

In [41]:
%%time

# Use KeyedVectors to load txt file
w2v = KeyedVectors.load_word2vec_format("../../embeddings/enwiki_20180420_100d.txt")

CPU times: user 5min 45s, sys: 15.6 s, total: 6min
Wall time: 6min 13s


In [46]:
# How many unique tokens are in our ACY dataset
print("Token Count: ", len(x_np), ", Unique Tokens Count: ", len(np.unique(x_np)))

Token Count:  17805 , Unique Tokens Count:  4142


In [60]:
# How many page titles do we have in our KWNLP dataset
print("Page Count: ", len(article_df['page_title']))

Page Count:  6189965


In [73]:
%%time
# Create single token similarity matrix to assess computation time

# Prepare blank matrix
w2v_similarity_matrix = []
page_titles_not_found = 0
for token in x_np[:1]:
    print("Token: ", token)
    for page_title in article_df['page_title'][:20]:
        print("Page Title: ", page_title)
        try:
            token_title_similarity = w2v.similarity(token, page_title)
            print("Token-Title Similarity: ", round(token_title_similarity, 5))
        except KeyError:
            page_titles_not_found += 1
print("******************************************************")
print("Page Titles Not In Wikipedia2Vec Vocabulary: ", page_titles_not_found)
print("******************************************************")

Token:  german
Page Title:  Anarchism
Page Title:  Autism
Page Title:  Albedo
Page Title:  A
Page Title:  Alabama
Page Title:  Achilles
Page Title:  Abraham_Lincoln
Page Title:  Aristotle
Page Title:  An_American_in_Paris
Page Title:  Academy_Award_for_Best_Production_Design
Page Title:  Academy_Awards
Page Title:  Actrius
Page Title:  Animalia_(book)
Page Title:  International_Atomic_Time
Page Title:  Altruism
Page Title:  Ayn_Rand
Page Title:  Alain_Connes
Page Title:  Allan_Dwan
Page Title:  Algeria
Page Title:  List_of_Atlas_Shrugged_characters
******************************************************
Page Titles Not In Wikipedia2Vec Vocabulary:  20
******************************************************
CPU times: user 3.59 ms, sys: 8.42 ms, total: 12 ms
Wall time: 20.6 ms


#### It appears impossible to match page titles to words in the Wikipedia2Vec vocabulary.

In [127]:
%%time
# Create single token similarity matrix to assess computation time
match_pct = 1

# Prepare blank matrix
w2v_similarity_dict = {}
anchor_texts_not_found = 0

# Swap between tqdm depending on if you want progress bar
for token in tqdm(x_np[:int(np.ceil(len(x_np)*0.05))]):
# for token in x_np[:int(np.ceil(len(x_np)*0.01))]:

#     print("Token: ", token)
    anchor_text_dict = {}
    
# Swap between tqdm depending on if you want progress bar
#     for anchor_text in tqdm(at_count_df['normalized_anchor_text'][:int(np.ceil(len(at_count_df)*(match_pct/100)))]):
    for anchor_text in at_count_df['normalized_anchor_text'][:int(np.ceil(len(at_count_df)*(match_pct/100)))]:
#         print("Anchor Text: ", anchor_text)
        try:
            token_title_similarity = w2v.similarity(token, anchor_text)
#             print("Token-Title Similarity: ", round(token_title_similarity, 5))
            anchor_text_dict[anchor_text] = token_title_similarity
        except KeyError:
            anchor_texts_not_found += 1
#             print(KeyError, "AT Not in Vocabulary")
    try:
        matched_value = max(anchor_text_dict, key=anchor_text_dict.get)
        matched_score = anchor_text_dict[matched_value]
    except ValueError:
        matched_value = None
        matched_score = 0.
    print("Token: ", token)
    print("Token Matched Value: ", matched_value, round(matched_score,5))
    w2v_similarity_dict[token] = (matched_value, matched_score)
print("******************************************************")
print("Anchor Texts Not In Wikipedia2Vec Vocabulary: ", anchor_texts_not_found)
print("******************************************************")

  0%|          | 1/891 [00:04<1:02:50,  4.24s/it]

Token:  german
Token Matched Value:  german 1.0


  0%|          | 2/891 [00:06<52:04,  3.51s/it]  

Token:  british
Token Matched Value:  british 1.0


  0%|          | 3/891 [00:07<42:53,  2.90s/it]

...

Token:  philippoussis
Token Matched Value:  halep 0.69997


100%|█████████▉| 890/891 [33:10<00:30, 30.65s/it]

Token:  australia
Token Matched Value:  australia 1.0


100%|██████████| 891/891 [33:12<00:00,  2.24s/it]

Token:  marc
Token Matched Value:  alex 0.74432
******************************************************
Anchor Texts Not In Wikipedia2Vec Vocabulary:  110861165
******************************************************
CPU times: user 22min 41s, sys: 46.6 s, total: 23min 28s
Wall time: 33min 12s


In [128]:
# Display generated dict
w2v_similarity_dict

{'german': ('german', 1.0),
 'british': ('british', 0.99999994),
 'brussels': ('brussels', 1.0),
 'european': ('european', 1.0),
 'commission': ('commission', 1.0),
 'germany': ('germany', 1.0),
 'union': ('union', 1.0),
 'britain': ('britain', 1.0),
 'franz': ('ludwig', 0.83037525),
 'fischler': ('bender', 0.6348997),
 'france': ('france', 1.0),
 'bse': ('bse', 1.0),
 'spanish': ('spanish', 1.0),
 'loyola': ('college', 0.71429205),
 'de': ('de', 1.0),
 'palacio': ('madrid', 0.7687936),
 'europe': ('europe', 1.0),
 'bonn': ('bonn', 1.0),
 'hendrix': ('hendrix', 1.0),
 'london': ('london', 1.0),
 'u.s.': (None, 0.0),
 'jimi': ('jimi', 1.0),
 'english': ('english', 1.0),
 'nottingham': ('nottingham', 1.0),
 'australian': ('australian', 1.0),
 'china': ('china', 1.0),
 'taiwan': ('taiwan', 1.0),
 'beijing': ('beijing', 1.0),
 'taipei': ('taipei', 1.0),
 'strait': ('straits', 0.72950417),
 'ukraine': ('ukraine', 1.0),
 'taiwanese': ('taiwanese', 1.0),
 'lien': ('ching', 0.6176003),
 'chan'

In [118]:
# What is the word it matched?
max_key = max(w2v_similarity_dict, key=w2v_similarity_dict.get)
print(max_key, w2v_similarity_dict[max_key])

german ('german', 1.0)


We see better success at matching to anchor text but matching this to an entity is still reliant on previously made connections.

In [ ]:
# Create text-to-page-to-title dataframe (as predictions)
preds_wiki2vec = pd.DataFrame({'mention': x_np, 
                             'entity_page_id': None,
                             'target_page_title': None
                            })

# tqdm is a progress bar package
# For all anchor texts, update with predictions from wiki2vec
for i in tqdm(range(len(preds_wiki2vec))):
    try:
        preds_wiki2vec.iloc[i, 1:3] = max_anchor_links.loc[preds_wiki2vec.iloc[i, 0], ['target_page_id', 'target_page_title']].values
    except KeyError:
        # Leave at default None values
        continue